<a href="https://colab.research.google.com/github/mmilannaik/BigOCheatSheet/blob/master/MA_Price_Elasticity_Catboost_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [14]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.4 MB/s eta 0:00:00


In [18]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [2]:
!apt-get update -qq
!apt-get install -y p7zip-full


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [3]:
!7z x "stores.csv.7z" -o"./extracted"



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,96 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 648 bytes (1 KiB)

Extracting archive: stores.csv.7z
--
Path = stores.csv.7z
Type = 7z
Physical Size = 648
Headers Size = 130
Method = LZMA2:12
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       1387
Compressed: 648


In [4]:
!7z x "items.csv.7z" -o"./extracted"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,96 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 14315 bytes (14 KiB)

Extracting archive: items.csv.7z
--
Path = items.csv.7z
Type = 7z
Physical Size = 14315
Headers Size = 122
Method = LZMA2:17
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       101841
Compressed: 14315


In [5]:
items = pd.read_csv('/content/extracted/items.csv')
stores = pd.read_csv('/content/extracted/stores.csv')

In [6]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [7]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
!7z x "train.csv.7z" -o"./extracted"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,96 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 474092593 bytes (453 MiB)

Extracting archive: train.csv.7z
--
Path = train.csv.7z
Type = 7z
Physical Size = 474092593
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      0% - train.csv                  1% - train.csv                  2% - train.csv                  3% - train.csv                  4% - train.csv                  5% - train.csv                  6% - train.csv                  7% - train.csv                  8% - train.csv

In [11]:
dtypes = { 'store_nbr':'int16','item_nbr':'int32','unit_sales':'float32' }
parse_dates=['date']
sales = pd.read_csv('/content/extracted/train.csv', usecols=['date','store_nbr','item_nbr','unit_sales','onpromotion'],
                    dtype=dtypes, parse_dates=parse_dates)

<ipython-input-11-3681241985>:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv('/content/extracted/train.csv', usecols=['date','store_nbr','item_nbr','unit_sales','onpromotion'],


In [12]:
!pip install dask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [10]:
import dask.dataframe as dd
sales = dd.read_csv('train.csv', dtype=dtypes, parse_dates=['date'])

ModuleNotFoundError: No module named 'dask'

# Modelling

In [16]:
# Merge datasets
data = sales.merge(items, on='item_nbr').merge(stores, on='store_nbr')

# Step 2: Feature engineering
from datetime import datetime
# compute discount_depth if price columns available
# placeholder logic:
data['day_of_week'] = data['date'].dt.dayofweek



KeyError: "['rolling_7'] not in index"

In [17]:

# Additional features: rolling mean sales, promo lag
data['rolling_7'] = data.groupby(['store_nbr','item_nbr'])['unit_sales'].transform(lambda x: x.shift().rolling(7).mean())

# Step 3: Train/test split
X = data[['store_nbr','item_nbr','onpromotion','day_of_week','rolling_7']]
y = data['unit_sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
# Step 4: CatBoost model
dtrain = Pool(X_train, y_train, cat_features=['store_nbr','item_nbr','onpromotion','day_of_week'])
model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=6)
model.fit(dtrain, eval_set=(X_test, y_test), verbose=100)

# Step 5: SHAP analysis
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.dependence_plot('onpromotion', shap_values, X_test)

CatBoostError: Invalid type for cat_feature[non-default value idx=4,feature_idx=2]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.